In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [ ]:
!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def generate_matches(data, entity_type, top_n=5):
    """
    Genera coincidencias entre universidades, empresas y entidades gubernamentales
    basadas en similitud de contenido utilizando TF-IDF y similitud coseno.
    """

    # Convertir datos en DataFrame
    df = pd.DataFrame(data)

    # Verificar si el tipo de entidad es válido
    if entity_type not in ['empresa', 'universidad', 'gobierno']:
        raise ValueError("El tipo de entidad debe ser 'empresa', 'universidad' o 'gobierno'")

    # Filtrar por el tipo de entidad
    target_entities = df[df['tipo'] == entity_type]
    other_entities = df[df['tipo'] != entity_type]

    # Vectorización TF-IDF
    vectorizer = TfidfVectorizer(stop_words=None)  # Cambio 'spanish' por None
    tfidf_matrix = vectorizer.fit_transform(df['descripcion'])

    # Calcular similitud coseno
    cosine_sim = cosine_similarity(tfidf_matrix)

    # Generar recomendaciones
    results = {}
    for index, row in target_entities.iterrows():
        idx = df[df['nombre'] == row['nombre']].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = [s for s in sim_scores if s[0] != idx]  # Excluir la misma entidad

        matched_entities = [(df.iloc[i]['nombre'], df.iloc[i]['tipo'], score) for i, score in sim_scores[:top_n]]
        results[row['nombre']] = matched_entities

    return results


In [ ]:
df = pd.DataFrame(data)
print(df['descripcion'])

0     Investigación en semiconductores y nanotecnolo...
1     Fabricación de dispositivos electrónicos y cir...
2     Programas de financiamiento para innovación en...
3     Centro de desarrollo en telecomunicaciones y e...
4     Diseño de sensores electrónicos para aplicacio...
5     Subsidios y apoyo a empresas de tecnología ava...
6     Investigación en inteligencia artificial aplic...
7     Desarrollo de software embebido para hardware ...
8     Normativas y regulaciones para la importación ...
9     Laboratorio de electrónica de potencia y energ...
10    Fabricación de baterías de alto rendimiento pa...
11    Fondos de inversión para startups de tecnologí...
12    Investigación en bioelectrónica y dispositivos...
13    Automatización de procesos industriales con si...
Name: descripcion, dtype: object


In [ ]:
def find_relevant_agents(data, query, top_n=5):
    """
    Encuentra los agentes (empresas, universidades, programas gubernamentales) más relevantes
    basados en una palabra clave o frase.
    """
    df = pd.DataFrame(data)

    # Vectorización TF-IDF
    vectorizer = TfidfVectorizer(stop_words=None)
    tfidf_matrix = vectorizer.fit_transform(df['descripcion'])
    query_vector = vectorizer.transform([query])

    # Calcular similitud coseno entre la consulta y todas las entidades
    cosine_sim = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Obtener las entidades más relevantes
    top_indices = cosine_sim.argsort()[-top_n:][::-1]
    relevant_agents = [(df.iloc[i]['nombre'], df.iloc[i]['tipo'], cosine_sim[i]) for i in top_indices]

    return relevant_agents

In [ ]:

# Datos de prueba ampliados
data = [
    {"nombre": "Universidad A", "tipo": "universidad", "descripcion": "Investigación en semiconductores y nanotecnología."},
    {"nombre": "Empresa B", "tipo": "empresa", "descripcion": "Fabricación de dispositivos electrónicos y circuitos integrados."},
    {"nombre": "Gobierno C", "tipo": "gobierno", "descripcion": "Programas de financiamiento para innovación en electrónica."},
    {"nombre": "Universidad D", "tipo": "universidad", "descripcion": "Centro de desarrollo en telecomunicaciones y electrónica industrial."},
    {"nombre": "Empresa E", "tipo": "empresa", "descripcion": "Diseño de sensores electrónicos para aplicaciones médicas."},
    {"nombre": "Gobierno F", "tipo": "gobierno", "descripcion": "Subsidios y apoyo a empresas de tecnología avanzada."},
    {"nombre": "Universidad G", "tipo": "universidad", "descripcion": "Investigación en inteligencia artificial aplicada a circuitos electrónicos."},
    {"nombre": "Empresa H", "tipo": "empresa", "descripcion": "Desarrollo de software embebido para hardware electrónico."},
    {"nombre": "Gobierno I", "tipo": "gobierno", "descripcion": "Normativas y regulaciones para la importación de semiconductores."},
    {"nombre": "Universidad J", "tipo": "universidad", "descripcion": "Laboratorio de electrónica de potencia y energías renovables."},
    {"nombre": "Empresa K", "tipo": "empresa", "descripcion": "Fabricación de baterías de alto rendimiento para dispositivos electrónicos."},
    {"nombre": "Gobierno L", "tipo": "gobierno", "descripcion": "Fondos de inversión para startups de tecnología electrónica."},
    {"nombre": "Universidad M", "tipo": "universidad", "descripcion": "Investigación en bioelectrónica y dispositivos médicos."},
    {"nombre": "Empresa N", "tipo": "empresa", "descripcion": "Automatización de procesos industriales con sistemas electrónicos."}
]


In [ ]:

# Ejemplo de uso
matches = generate_matches(data, 'empresa')
for key, value in matches.items():
    print(f"\n{key} se puede conectar con:")
    for v in value:
        print(f"  - {v[0]} ({v[1]}) con similitud {v[2]:.2f}")



Empresa B se puede conectar con:
  - Empresa K (empresa) con similitud 0.45
  - Universidad G (universidad) con similitud 0.27
  - Universidad M (universidad) con similitud 0.16
  - Empresa E (empresa) con similitud 0.13
  - Empresa N (empresa) con similitud 0.13

Empresa E se puede conectar con:
  - Empresa K (empresa) con similitud 0.20
  - Empresa B (empresa) con similitud 0.13
  - Gobierno L (gobierno) con similitud 0.13
  - Gobierno C (gobierno) con similitud 0.11
  - Empresa N (empresa) con similitud 0.11

Empresa H se puede conectar con:
  - Universidad D (universidad) con similitud 0.18
  - Gobierno L (gobierno) con similitud 0.13
  - Empresa K (empresa) con similitud 0.12
  - Gobierno C (gobierno) con similitud 0.11
  - Empresa E (empresa) con similitud 0.10

Empresa K se puede conectar con:
  - Empresa B (empresa) con similitud 0.45
  - Empresa E (empresa) con similitud 0.20
  - Gobierno L (gobierno) con similitud 0.18
  - Gobierno C (gobierno) con similitud 0.13
  - Univers

In [ ]:
# Buscar agentes relevantes por palabra clave
query = "desarrollo de baterías de litio"
relevant_agents = find_relevant_agents(data, query)
print("\nAgentes relevantes para la búsqueda de 'desarrollo de baterías de litio':")
for agent in relevant_agents:
    print(f"  - {agent[0]} ({agent[1]}) con similitud {agent[2]:.2f}")



Agentes relevantes para la búsqueda de 'desarrollo de baterías de litio':
  - Empresa K (empresa) con similitud 0.45
  - Universidad D (universidad) con similitud 0.32
  - Empresa H (empresa) con similitud 0.30
  - Universidad J (universidad) con similitud 0.19
  - Gobierno L (gobierno) con similitud 0.19
